# Imports

In [ ]:
import time
import urllib3
import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras import layers

In [ ]:
urllib3.disable_warnings()
tfds.disable_progress_bar()
tf.enable_eager_execution()

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)

# Load Cat vs Dog Dataset

In [ ]:
(training, validation), info = 

In [ ]:
def format_image(image, label):
    
    return image, label

num_examples =  
print("Number of examples:", num_examples)

In [ ]:
# mobilenet expects 224x224 images
BATCH = 32
IMAGE = 224

# Image Input Pipeline

In [ ]:
train_batches = 
validate_batches = 

# Define Feature Extractor

In [ ]:
url = 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'
feature_extractor = 

# Freeze Base Model

# Add Output Layer

In [ ]:
model = tf.keras.Sequential([
    
])

In [ ]:
model.summary()

# Train Model

In [ ]:
model.compile(
    
)

EPOCHS = 3

In [ ]:
log = model.fit( )

# Check Predictions

In [ ]:
class_names =  
print(class_names)

In [ ]:
image_batch, label_batch = 
image_batch = 
label_batch = 

In [ ]:
predictions = 
predictions = 
prediction_labels = 
prediction_names = 
print(prediction_names)

In [ ]:
print("Actual   : ", label_batch)
print("Predicted: ", prediction_labels)

In [ ]:
plt.figure(figsize=(10,9))
for n in range(30):
    

# Save .h5 Model

In [ ]:
t = time.time()
export_path = "./cat_dog_{}.h5".format(int(t))
print(export_path)

In [ ]:
model.save(export_path)

# Load .h5 Model 

In [ ]:
reload = 

In [ ]:
reload.summary()

# Confirm Equality

In [ ]:
results = 
reload_results = 
(abs(results - reload_results)).max()

# Continue Training

In [ ]:
EPOCHS = 3
log = reload.fit( )

# Export SavedModel

In [ ]:
t = time.time()
model_export_path = "./cat_dog_{}".format(int(t))
print(export_path)

In [ ]:
tf.saved_model.save(reload, model_export_path)

In [ ]:
!ls {model_export_path}

# Load SavedModel

In [ ]:
loaded = 

In [ ]:
loaded_result_batch = 

In [ ]:
(abs(results - loaded_result_batch)).max()

# Load SavedModel as Keras Model

In [ ]:
saved_model = 

In [ ]:
model_results = 
saved_model_results = 

In [ ]:
(abs(model_results - saved_model_results)).max()

# Download Model

In [ ]:
!zip -r model.zip {model_export_path}